# Packages

In [ ]:
import numpy as np
from pandas import DataFrame
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import sklearn 
import sspa
import sspa.utils
import gseapy.plot as gp
import networkx
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats
import statsmodels.api as sm
import plotly.graph_objects as go
import plotly.express as px
import urllib.request
import statsmodels
import networkx as nx
import math
import itertools 
from scipy.stats import hypergeom as hg
import textwrap
from itertools import chain
import missforest
import pathintegrate


# A) Loading in data - HERE WE ARE PERFORMING PATHINTEGRATE SUPERVISED AT DIFFERENT THRESHOLDs

### Raw metabolomics data (this is used for generating different ID - converted matrices)

In [ ]:
# Reading in the metabolomics data
metabolomics_data_processed = pd.read_csv('/Users/judepops/Documents/PathIntegrate/Code/Processing/Processing_Cleaned/cleaned_metabolomics_data_covid.csv')
metabolomics_data_processed = metabolomics_data_processed.set_index('sample_id')
metabolomics_data_processed_final = metabolomics_data_processed.iloc[:, :-7]
metabolomics_data_processed_final.columns = [col.strip().lower() for col in metabolomics_data_processed_final.columns]

last_7_columns = metabolomics_data_processed.iloc[:, -7:]
last_7_columns


### llm ID predictions: subset to metabolites that have manual annotations

In [ ]:
llm_subset = pd.read_csv('/Users/judepops/Documents/PathIntegrate/Code/LLM_Annotation/manual_automated_subset.csv', index_col=0)

### Generating llm predictions at different threshold cutoffs: 0.5 to 1.0

In [ ]:
# clarify the thresohld columns
threshold_columns = [col for col in llm_subset.columns if 'Matched COMPOUND_ID' in col]

# unpivot the dataframe using melting
llm_melted = pd.melt(llm_subset, 
                    id_vars=['Query'],
                    value_vars=threshold_columns,
                    var_name='Threshold',
                    value_name='ChEBI')

# cleaning the threshold column - removing any whitespace that can cause spurious matching
llm_melted['Threshold'] = llm_melted['Threshold'].str.extract(r'(\d+\.\d+)$').astype(float)
llm_melted = llm_melted.dropna(subset=['Threshold'])
llm_melted.reset_index(inplace=True, drop=True)

# removing any columns that dont have chebi ID predictions (na columns)
llm_melted = llm_melted.dropna(subset=['ChEBI'])

# making chebi correct
llm_melted['ChEBI'] = pd.to_numeric(llm_melted['ChEBI'], errors='coerce')
llm_melted['ChEBI'] = llm_melted['ChEBI'].astype('Int64')

# getting unique threshold values
unique_thresholds = llm_melted['Threshold'].unique()

# now this is a dciontary to hold the individual dataframes created by each threshold subset
threshold_dfs = {}
for threshold in unique_thresholds:
    threshold_df = llm_melted[llm_melted['Threshold'] == threshold].copy()
    threshold_dfs[f"llm_{threshold}"] = threshold_df

# printing the keys to visualise these dataframes
print(threshold_dfs.keys())


### Now we map the raw metabolomics data to ChEBI IDs using the different threshold prediction dataframes as the conversion reference

### multiple ID - converted matrices are created (each with a differnt number of IDs due to differetn LLM thresholds)

In [ ]:
# loading the downloaded multi-omics reactome pathway database 
mo_paths_reactome = sspa.process_gmt(infile='Reactome_Homo_sapiens_pathways_multiomics_R89.gmt')

# converign the dataframe to object type
mo_paths_reactome = mo_paths_reactome.astype('object')

# intialising a dictionary to hold  processed data mapped for each threshold
processed_data_mapped_dict = {}

# looping through each threshold dataframe
for threshold, df in threshold_dfs.items():
    # running provided commands on each dataframe
    df = df.drop(columns='Threshold')
    processed_data_mapped = sspa.map_identifiers(df, output_id_type="ChEBI", matrix=metabolomics_data_processed_final)
    processed_data_mapped.columns = processed_data_mapped.columns.map(str)
    
    all_reactome_cpds = set(sum(sspa.utils.pathwaydf_to_dict(mo_paths_reactome).values(), []))
    mapped_annotated_cpds_met = set(processed_data_mapped.columns) & set(all_reactome_cpds)

    last_7_columns = metabolomics_data_processed.iloc[:, -7:]
    processed_data_mapped_final = pd.concat([processed_data_mapped, last_7_columns], axis=1)
    
    # storing the mapped processed data as a dictionary
    processed_data_mapped_dict[threshold] = processed_data_mapped_final

    
# subsetting the dictionary to only include thresholds that are 0,5 or greater (there is no change in compounds before 0.5)
subset_processed_data = {threshold: data for threshold, data in processed_data_mapped_dict.items() if float(threshold.split('_')[-1]) >= 0.5}

# printing the contents of the datafame to verify
for threshold, data in subset_processed_data.items():
    print(f"Threshold: {threshold}, Data shape: {data.shape}")

# Fitting a PathIntegrate cross-validated single-view model to each threshold ID-converted matrix

### The logistic regression function is chosen, aswell as the SVD ssPA method. Pathways are only used at a minimum of 4 compounds per pathway

### This code takes around 3 hours to run

In [ ]:
# importing key dependencies for the machine learnign model
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from scipy import interpolate

# THIS IS A CONDENSED VERSION OF THE CODE AVAILABLE IN SECTION 2 STANDARD PATHINTEGRATE WORKFLOW - 
# THIS IS JUST A FUNCTION TO RUN THE SINGLEVIEW LOGISTIC REGRESSION CROSS-VALIDATED MODEL ON EVER 
# SINGLE DIFFERENT ID-CONVERTED MATRXI IN ORDER TO DETERMINE THE BEST ID-CONVERTED MATRIX FOR PATINET CLASSIFICATION


# FUNCITON 1: CREATING CROSS-VALIDATED MODEL (SEE SECTION 2)

def train_and_evaluate_model(random_seed, prot, metab, mo_paths, shuffle_labels=False):
    X_train_prot, X_test_prot, y_train, y_test = train_test_split(
        prot.drop(columns=['Condition_Group']), prot['Condition_Group'],
        test_size=0.33, random_state=random_seed, stratify=prot['Condition_Group']
    )

    if shuffle_labels:
        np.random.shuffle(y_train.values)
        np.random.shuffle(y_test.values)

    X_train_met, X_test_met = metab.loc[X_train_prot.index, :], metab.loc[X_test_prot.index, :]

    pi_model = pathintegrate.PathIntegrate(
        omics_data={'Metabolomics_train': X_train_met, 'Proteomics_train': X_train_prot},
        metadata=y_train,
        pathway_source=mo_paths,
        sspa_scoring=sspa.sspa_SVD,
        min_coverage=6
    )

    cv_single_view = pi_model.SingleViewCV(
        LogisticRegression,
        model_params={'random_state': 0, 'max_iter': 500},
        cv_params={'cv': 5, 'scoring': 'f1', 'verbose': 2}
    )

    print('Mean cross-validated F1 score: ', np.mean(cv_single_view))

    sv_tuned = pi_model.SingleView(
        model=LogisticRegression,
        model_params={'C': 21.54434690031882, 'random_state': 0, 'max_iter': 500}
    )

    concat_data = pd.concat({'Metabolomics_test': X_test_met, 'Proteomics_test': X_test_prot.iloc[:, :-1]}.values(), axis=1)

    pipe_sv = Pipeline([
        ('Scaler', StandardScaler().set_output(transform="pandas")),
        ('sspa', pi_model.sspa_method(pi_model.pathway_source, pi_model.min_coverage)),
    ])

    test_set_scores = pipe_sv.fit_transform(concat_data)

    sv_pred = sv_tuned.predict(test_set_scores)
    sv_pred_prob = sv_tuned.predict_proba(test_set_scores)[:, 1]

    test_set_f1 = f1_score(y_test, sv_pred)
    test_set_precision = precision_score(y_test, sv_pred)
    test_set_recall = recall_score(y_test, sv_pred)
    test_set_auc = roc_auc_score(y_test, sv_pred_prob)

    fpr, tpr, _ = roc_curve(y_test, sv_pred_prob)

    return test_set_f1, test_set_precision, test_set_recall, test_set_auc, fpr, tpr, random_seed


# FUNCTION 2: RUNNIGN THE MODEL FOR 50 DIFFERENT RANDOM SEEDS AND GETTING THE AVERAGE ROC 
# this function also saves key metrics to be used for plotting and also for evaluatign different matrices

def run_model_for_all_seeds(random_seeds, prot, metab, mo_paths, shuffle_labels=False):
    num_runs = len(random_seeds)
    f1_scores = []
    precision_scores = []
    recall_scores = []
    auc_scores = []
    fpr_list = []
    tpr_list = []
    all_fpr = np.linspace(0, 1, 100)

    for i in range(num_runs):
        random_seed = random_seeds[i]
        f1, precision, recall, auc, fpr, tpr, used_seed = train_and_evaluate_model(random_seed, prot, metab, mo_paths, shuffle_labels)
        f1_scores.append(f1)
        precision_scores.append(precision)
        recall_scores.append(recall)
        auc_scores.append(auc)
        fpr_list.append(fpr)
        tpr_list.append(interpolate.interp1d(fpr, tpr)(all_fpr))
        print(f"Run {i + 1}: F1 = {f1}, Precision = {precision}, Recall = {recall}, AUC = {auc}, Seed = {used_seed}")

    mean_tpr = np.mean(tpr_list, axis=0)
    std_tpr = np.std(tpr_list, axis=0)
    mean_auc = np.mean(auc_scores)
    std_auc = np.std(auc_scores)

    return all_fpr, mean_tpr, mean_auc, std_tpr, std_auc

# generating 50 random seeds to run the model
random_seeds = np.random.randint(200, size=50)

# loading the pathway data for Reactome - chebi IDs map to the reactome pathway database

mo_paths_reactome = sspa.process_gmt(infile='Reactome_Homo_sapiens_pathways_multiomics_R89.gmt')
mo_paths_reactome = mo_paths_reactome.astype('object')

# the function to process datasets and run models

def process_and_run_model(metab, prot_file, mo_paths, random_seeds):
    prot = pd.read_csv(prot_file)
    prot.set_index('sample_id', inplace=True)
    metab.set_index('sample_id', inplace=True)
    prot = prot.drop(columns=['Who', 'Race', 'Age', 'Group', 'Age_Group', 'Race_Group'])
    metab = metab.drop(columns=['Who', 'Race', 'Age', 'Group', 'Age_Group', 'Race_Group'])
    common_indices = prot.index.intersection(metab.index)
    prot = prot.loc[common_indices]
    metab = metab.loc[common_indices]
    metab = metab.iloc[:, :-1]
    prot['Condition_Group'] = prot['Condition_Group'].map({'Severe': 1, 'Mild': 0})
    
    return run_model_for_all_seeds(random_seeds, prot, metab, mo_paths)

# proteomics data pathway to be loaded in and integrate diwht the different metabolomics data thresholds
prot_file = '/Users/judepops/Documents/PathIntegrate/Code/Pathway_Analysis/COVID_Pro_UniProt_Final.csv'

# these are the thresholds to process - we didnt want to run all of them as this took too logn and the lower thresholds 
# are uninformative as they will just yield the same results

selected_thresholds = ['llm_0.6', 'llm_0.7', 'llm_0.75', 'llm_0.8', 'llm_0.9', 'llm_1.0']

# dictionary to hold the results for each selected threshold - we can check this later
results_dict = {}

# looping through each version of metabolomics data in the processed_data_mapped_dict dictionary we created earlier
for threshold, data in subset_processed_data.items():
    if threshold in selected_thresholds:
        data.reset_index(inplace=True)
        
        # processing and running the model for each selected threshold
        all_fpr, mean_tpr, mean_auc, std_tpr, std_auc = process_and_run_model(data, prot_file, mo_paths_reactome, random_seeds)
        
        # storing the results
        results_dict[threshold] = {
            'all_fpr': all_fpr,
            'mean_tpr': mean_tpr,
            'mean_auc': mean_auc,
            'std_tpr': std_tpr,
            'std_auc': std_auc
        }

# plotting ROC curves
plt.figure()

# creating distrinct colours with seaborn
colors = sns.color_palette("hsv", len(results_dict))

# Loop through the results and plot each one
for i, (threshold, results) in enumerate(results_dict.items()):
    plt.plot(results['all_fpr'], results['mean_tpr'], color=colors[i], 
             label=f'Threshold {threshold} ROC curve (area = {results["mean_auc"]:.2f}, std = {results["std_auc"]:.2f})')
    plt.fill_between(results['all_fpr'], results['mean_tpr'] - results['std_tpr'], 
                     results['mean_tpr'] + results['std_tpr'], color=colors[i], alpha=0.2)

plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('roc curves for diff thresholds with standard deviation')
plt.legend(loc="lower right", prop={'size': 8})
plt.show()


### creating and saving a plot with desired colours

In [ ]:
selected_thresholds = ['llm_0.6', 'llm_0.7', 'llm_0.75','llm_0.8', 'llm_0.9', 'llm_1.0']
custom_palette = ["red", "blue", "green", 'brown', "orange", "purple"]

# intiialising the figure
plt.figure()

# looping through the results for selected thresholds and plotting each one
for i, threshold in enumerate(selected_thresholds):
    results = results_dict[threshold]
    # ensuring the curve starts at (0, 0)
    if results['all_fpr'][0] != 0 or results['mean_tpr'][0] != 0:
        results['all_fpr'] = np.insert(results['all_fpr'], 0, 0)
        results['mean_tpr'] = np.insert(results['mean_tpr'], 0, 0)

    # plotting as a step plot
    plt.step(results['all_fpr'], results['mean_tpr'], where='post', color=custom_palette[i], 
             label=f'Threshold {threshold} ROC curve (area = {results["mean_auc"]:.2f}, std = {results["std_auc"]:.2f})')

plt.plot([0, 1], [0, 1], color='gray', linestyle='--')

# setting the labels and title
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('roc curves for diff thresholds with standard deviation')

plt.legend(loc="lower right", prop={'size': 6.5})
plt.savefig('/Users/judepops/Documents/PathIntegrate/Code/Final_Scripts/Results/Results_D/ROC_Threshold_Comparison/roc_curves_final_thresh.png', dpi=500)
plt.show()

# Supervised Analysis to Compare Methods - Not thresholds

### ALl of the dataframes used here were created in Section_3_ComparingMethods.ipynb

### This code is a similar ROC curve to before, with the same parameters but different input datasets!

In [ ]:
# same as last time - function to evaluate and train model
def train_and_evaluate_model(random_seed, prot, metab, mo_paths, shuffle_labels=False):
    X_train_prot, X_test_prot, y_train, y_test = train_test_split(
        prot.drop(columns=['Condition_Group']), prot['Condition_Group'],
        test_size=0.33, random_state=random_seed, stratify=prot['Condition_Group']
    )

    if shuffle_labels:
        np.random.shuffle(y_train.values)
        np.random.shuffle(y_test.values)

    X_train_met, X_test_met = metab.loc[X_train_prot.index, :], metab.loc[X_test_prot.index, :]

    pi_model = pathintegrate.PathIntegrate(
        omics_data={'Metabolomics_train': X_train_met, 'Proteomics_train': X_train_prot},
        metadata=y_train,
        pathway_source=mo_paths,
        sspa_scoring=sspa.sspa_SVD,
        min_coverage=6
    )

    cv_single_view = pi_model.SingleViewCV(
        LogisticRegression,
        model_params={'random_state': 0, 'max_iter': 500},
        cv_params={'cv': 5, 'scoring': 'f1', 'verbose': 2}
    )

    print('Mean cross-validated F1 score: ', np.mean(cv_single_view))

    sv_tuned = pi_model.SingleView(
        model=LogisticRegression,
        model_params={'C': 21.54434690031882, 'random_state': 0, 'max_iter': 500}
    )

    concat_data = pd.concat({'Metabolomics_test': X_test_met, 'Proteomics_test': X_test_prot.iloc[:, :-1]}.values(), axis=1)

    pipe_sv = Pipeline([
        ('Scaler', StandardScaler().set_output(transform="pandas")),
        ('sspa', pi_model.sspa_method(pi_model.pathway_source, pi_model.min_coverage)),
    ])

    test_set_scores = pipe_sv.fit_transform(concat_data)

    sv_pred = sv_tuned.predict(test_set_scores)
    sv_pred_prob = sv_tuned.predict_proba(test_set_scores)[:, 1]

    test_set_f1 = f1_score(y_test, sv_pred)
    test_set_precision = precision_score(y_test, sv_pred)
    test_set_recall = recall_score(y_test, sv_pred)
    test_set_auc = roc_auc_score(y_test, sv_pred_prob)

    fpr, tpr, _ = roc_curve(y_test, sv_pred_prob)

    return test_set_f1, test_set_precision, test_set_recall, test_set_auc, fpr, tpr, random_seed

# function to run the model for all seeds and calculate the average roc
def run_model_for_all_seeds(random_seeds, prot, metab, mo_paths, shuffle_labels=False):
    num_runs = len(random_seeds)
    f1_scores = []
    precision_scores = []
    recall_scores = []
    auc_scores = []
    fpr_list = []
    tpr_list = []
    all_fpr = np.linspace(0, 1, 100)

    for i in range(num_runs):
        random_seed = random_seeds[i]
        f1, precision, recall, auc, fpr, tpr, used_seed = train_and_evaluate_model(random_seed, prot, metab, mo_paths, shuffle_labels)
        f1_scores.append(f1)
        precision_scores.append(precision)
        recall_scores.append(recall)
        auc_scores.append(auc)
        fpr_list.append(fpr)
        tpr_list.append(interpolate.interp1d(fpr, tpr)(all_fpr))
        print(f"Run {i + 1}: F1 = {f1}, Precision = {precision}, Recall = {recall}, AUC = {auc}, Seed = {used_seed}")

    mean_tpr = np.mean(tpr_list, axis=0)
    std_tpr = np.std(tpr_list, axis=0)
    mean_auc = np.mean(auc_scores)
    std_auc = np.std(auc_scores)

    return all_fpr, mean_tpr, mean_auc, std_tpr, std_auc

# generating 50 random seeds to run the model
random_seeds = np.random.randint(200, size=50)

# Common pathway data for Reactome
mo_paths_reactome = sspa.process_gmt(infile='Reactome_Homo_sapiens_pathways_multiomics_R89.gmt')
mo_paths_reactome = mo_paths_reactome.astype('object')

# function to process datasets and run models prior to the cross valiation adn model
def process_and_run_model(metab, prot_file, mo_paths, random_seeds, method_name):
    prot = pd.read_csv(prot_file)
    prot.set_index('sample_id', inplace=True)
    metab.set_index('sample_id', inplace=True)
    prot = prot.drop(columns=['Who', 'Race', 'Age', 'Group', 'Age_Group', 'Race_Group'])
    metab = metab.drop(columns=['Who', 'Race', 'Age', 'Group', 'Age_Group', 'Race_Group'])
    common_indices = prot.index.intersection(metab.index)
    prot = prot.loc[common_indices]
    metab = metab.loc[common_indices]
    metab = metab.iloc[:, :-1]
    prot['Condition_Group'] = prot['Condition_Group'].map({'Severe': 1, 'Mild': 0})
    
    return run_model_for_all_seeds(random_seeds, prot, metab, mo_paths)

# loading and processing versions of metabolomics data
metab_manual = pd.read_csv('/Users/judepops/Documents/PathIntegrate/Code/Final_Scripts/Results/Results_D/COVID_Met_ChEBI_Maual.csv')
metab_llm = pd.read_csv('/Users/judepops/Documents/PathIntegrate/Code/Final_Scripts/Results/Results_D/COVID_Met_ChEBI_LLM_0.75.csv')
metab_met = pd.read_csv('/Users/judepops/Documents/PathIntegrate/Code/Final_Scripts/Results/Results_D/COVID_Met_ChEBI_Metaboanalyst.csv')
metab_llm_v2 = pd.read_csv('/Users/judepops/Documents/PathIntegrate/Code/Final_Scripts/Results/Results_D/COVID_Met_ChEBI_LLM_V2.csv')
prot_file = '/Users/judepops/Documents/PathIntegrate/Code/Pathway_Analysis/COVID_Pro_UniProt_Final.csv'

print('Starting')

# running the mdoel for each dataset
fpr_manual, mean_tpr_manual, mean_auc_manual, std_tpr_manual, std_auc_manual = process_and_run_model(metab_manual, prot_file, mo_paths_reactome, random_seeds, 'Manual')
print('Done Manual')
fpr_llm, mean_tpr_llm, mean_auc_llm, std_tpr_llm, std_auc_llm = process_and_run_model(metab_llm, prot_file, mo_paths_reactome, random_seeds, 'LLM')
print('Done LLM')
fpr_met, mean_tpr_met, mean_auc_met, std_tpr_met, std_auc_met = process_and_run_model(metab_met, prot_file, mo_paths_reactome, random_seeds, 'Metaboanalyst')
print('Done Met')
fpr_llm_v2, mean_tpr_llm_v2, mean_auc_llm_v2, std_tpr_llm_v2, std_auc_llm_v2 = process_and_run_model(metab_llm_v2, prot_file, mo_paths_reactome, random_seeds, 'LLM V2')
print('Done LLM V2')

# plottign the roc curves
plt.figure()

plt.plot(fpr_manual, mean_tpr_manual, color='blue', label=f'Manual (area = {mean_auc_manual:.2f}, std = {std_auc_manual:.2f})')

plt.plot(fpr_llm, mean_tpr_llm, color='red', label=f'LLM 0.75 V1 (area = {mean_auc_llm:.2f}, std = {std_auc_llm:.2f})')

plt.plot(fpr_met, mean_tpr_met, color='green', label=f'Metaboanalyst (area = {mean_auc_met:.2f}, std = {std_auc_met:.2f})')

plt.plot(fpr_llm_v2, mean_tpr_llm_v2, color='magenta', label=f'LLM 0.75 V2 (area = {mean_auc_llm_v2:.2f}, std = {std_auc_llm_v2:.2f})')

plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for Different Metabolomics Methods with Standard Deviation')
plt.legend(loc="lower right", prop={'size': 8})
plt.show()


### creating a ROC curve plot 

In [ ]:
def ensure_start_at_zero(fpr, tpr):
    if fpr[0] != 0 or tpr[0] != 0:
        fpr = np.insert(fpr, 0, 0)
        tpr = np.insert(tpr, 0, 0)
    return fpr, tpr

plt.style.use('default')
plt.style.use('seaborn-v0_8-white')
plt.figure()


fpr_met, mean_tpr_met = ensure_start_at_zero(fpr_met, mean_tpr_met)
plt.step(fpr_met, mean_tpr_met, where='post', color='orange', label=f'Metaboanalyst (area = {mean_auc_met:.2f}, std = {std_auc_met:.2f})')
fpr_llm, mean_tpr_llm = ensure_start_at_zero(fpr_llm, mean_tpr_llm)
plt.step(fpr_llm, mean_tpr_llm, where='post', color='blue', label=f'0.75 LLM v1 (area = {mean_auc_llm:.2f}, std = {std_auc_llm:.2f})')
fpr_llm_v2, mean_tpr_llm_v2 = ensure_start_at_zero(fpr_llm_v2, mean_tpr_llm_v2)
plt.step(fpr_llm_v2, mean_tpr_llm_v2, where='post', color='red', label=f'0.75 LLM v2 (area = {mean_auc_llm_v2:.2f}, std = {std_auc_llm_v2:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')

plt.xlabel('fpr')
plt.ylabel('tpr')
plt.title('roc Curves for diff annotation methods')
plt.legend(loc="lower right", prop={'size': 9})

plt.tck_params(axis='both', which='major', labelsize=13)

#save
#plt.savefig('/Users/judepops/Documents/PathIntegrate/Code/Final_Scripts/Results/Results_D/ROC_Reactome_Method_Comparison/roc_curves_met_FINAL_NoLLMMet.png', dpi=500)

# Show the plot
plt.show()
